# **MCTS Algorithm**

**The MCTS Algorithm takes Inputs a Base Node, K Number of Simulations, Tie Factor, Final Selection.**

**Base Node contains the Board Position, Player who played the board recently and some other properties like children, Parents, Visits, Wins**

**K is the Number of Simulations to be run by the MCTS Algorithm**

**Tie Factor(Default = 0) is the value how much the MCTS Algorithm considers a Tie in the Game**

**Tie Factor can be thought of as giving a Reward also for a Tie Position**

**Final Selection is Based on Maximum Visits(Default) of a Child or Maximum Win Percent of a Child**

**If we are playing our MCTS Agent against a Hard Player, then we can keep the Tie Factor High so that the Agent can Tie instead of Losing(instead of trying to Win) to the Hard Player**

In [1]:
# MCTS Algorithm

# Importing Required Libraries
import numpy as np

# Board Positions
# 0 - Empty
# 5 - X (Agent)
# 7 - O (Opponent)

# MCTS Tree Node containing the properties children, board, player who played the board recently,
# Parents, Visits, Wins
class tree_node:
    def __init__(self,board_pos,player):
        self.children = []
        self.board = board_pos
        self.parent = None
        self.visits = 0
        self.wins = 0
        self.player = player # Either 5(Agent) or 7(Opp)

# Function MCTS which Does Monte Carlo Tree Search for a given Base Node, K (Number of Simulations), tie_factor, final_selection
def mcts(node: tree_node,K,tie_factor=0,final_selection="visits"):
    for k in range(K):
        #print("Simulation Number",k)
        dep = 0
        leaf,depth = traverse(node,dep)
        simresult = rollout(leaf.board,leaf.player)
        backpropagate(leaf,simresult,depth,tie_factor)
        #print(depth)
    
    # Selecting child with max number of visits
    if final_selection == "visits":
        best_val = -999
        best_child_ind = -1
        for i in range(len(node.children)):
            if node.children[i].visits > best_val:
                best_val = node.children[i].visits
                best_child_ind = i
        return node.children[best_child_ind]
    # Selecting child with max win_percent
    elif final_selection == "win_percent":
        best_val = -999
        best_child_ind = -1
        for i in range(len(node.children)):
            win_percent = node.children[i].wins/node.children[i].visits
            if win_percent > best_val:
                best_val = win_percent
                best_child_ind = i
        return node.children[best_child_ind]
    
# Function to Visualise the Board given a node
def visualise_node_board(node: tree_node):
    board = node.board
    for i in range(9):
        if board[i] == 0:
            print("|   |",end=" ")
        elif board[i] == 5:
            print("| X |",end=" ")
        else:
            print("| O |",end=" ")
        if i%3 == 2:
            print()

# Function to Check the win of Board given a node
def check_win(node: tree_node):
    b = node.board
    l = []
    for i in range(3):
        l.append(b[i]+b[i+3]+b[i+6])
        l.append(b[3*i]+b[3*i+1]+b[3*i+2])
    l.append(b[0]+b[4]+b[8])
    l.append(b[2]+b[4]+b[6])

    for i in l:
        if i == 15:
            #self.win = 1
            return 1 #"AGENT WIN"
        elif i == 21:
            #self.win = 0
            return 0 #"OPPONENT WIN"
    count = 0
    for i in b:
        if i != 0:
            count += 1
    if count == 9:
        return -2 #"TIE"
    return -1#"CONTINUE"

# Have to expand all the Children of the Node
def random_unexpanded_child(node: tree_node):
    empty_places = []
    for i in range(len(node.board)):
        if node.board[i] == 0:
            empty_places.append(i)
    next_player = 5
    if node.player == 5:
        next_player = 7
    for ind in empty_places:
        next_board = node.board.copy()
        next_board[ind] = next_player
        child = tree_node(next_board,next_player)
        child.parent = node
        node.children.append(child)
        child.parent = node
    return np.random.choice(node.children) # selected_child
    

def traverse(node: tree_node,depth):
    # Selection Stage
    while len(node.children) != 0:
        node = selection(node, 1.4)
        depth += 1
    if check_win(node) == -1: # Game is Continuing
        # Expansion Stage
        node = random_unexpanded_child(node)
        depth += 1
    return node,depth

# Selection Function which selects the Best Child given a node.
def selection(node: tree_node,C):
    ucb = np.zeros(len(node.children))
    unexplored_children = []
    #print("Parent Visits",node.visits)
    for i in range(len(node.children)):
        if node.children[i].visits == 0:
            unexplored_children.append(i)
        else:
            ucb[i] = node.children[i].wins/node.children[i].visits + C*np.sqrt(np.log(node.visits+1)/node.children[i].visits)
    if len(unexplored_children) > 0:
        selected_child = np.random.choice(unexplored_children)
        return node.children[selected_child]
    return node.children[np.argmax(ucb)]

# Function which checks the win of Board given a board
def check_win_board(board):
    b = board
    l = []
    for i in range(3):
        l.append(b[i]+b[i+3]+b[i+6])
        l.append(b[3*i]+b[3*i+1]+b[3*i+2])
    l.append(b[0]+b[4]+b[8])
    l.append(b[2]+b[4]+b[6])
    for i in l:
        if i == 15:
            return 1 #"AGENT WIN"
        elif i == 21:
            return 0 #"OPPONENT WIN"
    count = 0
    for i in b:
        if i != 0:
            count += 1
    if count == 9:
        return -2 #"TIE"
    return -1#"CONTINUE"

# Function which takes input a Board Position and Player Number and returns the Next Board, Next Player
def random_play(board_pos,player):
    empty_places = []
    for i in range(len(board_pos)):
        if board_pos[i] == 0:
            empty_places.append(i)
    ind = np.random.choice(empty_places)
    next_player = 5
    if player == 5:
        next_player = 7
    next_board = board_pos.copy()
    next_board[ind] = next_player
    return next_board, next_player

# Rollout a New Game starting from given Board Position and previous Player
def rollout(node_board,player):
    game_stop = check_win_board(node_board)
    if game_stop != -1:
        return game_stop
    else:
        child_board, player = random_play(node_board,player)
        return rollout(child_board,player)

# Back-Propagate the Sim Result to all the nodes starting the leaf Node to Parent Root Node
def backpropagate(node: tree_node,result,depth,tie_factor=1):
    for i in range(depth+1):
        if result == 1:
            if node.player == 5:
                node.wins += 1
        elif result == 0:
            if node.player == 7:
                node.wins += 1
        
        # This has to be adjusted for different players
        elif result == -2:
            node.wins += tie_factor
        node.visits += 1
        node = node.parent

# Random Opponent to be Played given the Board Position
def random_opponent(board):
    empty_places = []
    for i in range(len(board)):
        if board[i] == 0:
            empty_places.append(i)
    ind = np.random.choice(empty_places)
    opponent = 7
    next_board = board.copy()
    next_board[ind] = opponent
    return next_board

# Board Positions
# 0 - Empty
# 5 - X (Agent)
# 7 - O (Opponent)

print("Test Example")
# Opp - Center Square
game_board = [0,0,7,0,0,0,0,0,0]

base_node = tree_node(game_board,7)
print("The Initial Board is")
visualise_node_board(base_node)
best_node = mcts(base_node,1000)
print("The Board Position after MCTS Agent Move is")
visualise_node_board(best_node)


Test Example
The Initial Board is
|   | |   | | O | 
|   | |   | |   | 
|   | |   | |   | 
The Board Position after MCTS Agent Move is
|   | |   | | O | 
|   | | X | |   | 
|   | |   | |   | 


**MCTS agent is one move away from win**

In [2]:
# Board Positions
# 0 - Empty
# 5 - X (Agent)
# 7 - O (Opponent)

print("MCTS agent is one move away from win")
# Agent Winning Positions
game_boards = [[5,0,0,0,0,7,7,0,5],[5,5,0,0,7,0,7,0,0]]

# Testing when MCTS agent is one move away from win
for game_board in game_boards:
    base_node = tree_node(game_board,7)
    print("Starting Board Position is:")
    visualise_node_board(base_node)
    best_node = mcts(base_node,1000)
    print("The Board Position after MCTS Agent Move is")
    visualise_node_board(best_node)
    win = check_win(best_node)
    if win == 1:
        print("MCTS Agent Wins")
    elif win == 2:
        print("Opponent Wins")
    elif win == -2:
        print("The GAME is a TIE")
    else:
        print("GAME is continuing")
    print()

MCTS agent is one move away from win
Starting Board Position is:
| X | |   | |   | 
|   | |   | | O | 
| O | |   | | X | 
The Board Position after MCTS Agent Move is
| X | |   | |   | 
|   | | X | | O | 
| O | |   | | X | 
MCTS Agent Wins

Starting Board Position is:
| X | | X | |   | 
|   | | O | |   | 
| O | |   | |   | 
The Board Position after MCTS Agent Move is
| X | | X | | X | 
|   | | O | |   | 
| O | |   | |   | 
MCTS Agent Wins



**MCTS agent is one move away from loss**

In [3]:
# Board Positions
# 0 - Empty
# 5 - X (Agent)
# 7 - O (Opponent)

print("MCTS agent is one move away from loss")
# Opponent Winning Positions
game_boards = [[5,7,0,0,7,5,0,0,0],[5,7,0,0,0,5,0,7,0],[5,5,7,0,0,0,7,0,0],[5,5,7,0,7,0,0,0,0]]

# Testing when MCTS agent is one move away from Loss
for game_board in game_boards:
    base_node = tree_node(game_board,7)
    print("Starting Board Position is:")
    visualise_node_board(base_node)
    best_node = mcts(base_node,1000)
    print("The Board Position after MCTS Agent Move is")
    visualise_node_board(best_node)
    win = check_win(best_node)
    if win == 1:
        print("MCTS Agent Wins")
    elif win == 2:
        print("Opponent Wins")
    elif win == -2:
        print("The GAME is a TIE")
    else:
        print("GAME is continuing")
    print()

MCTS agent is one move away from loss
Starting Board Position is:
| X | | O | |   | 
|   | | O | | X | 
|   | |   | |   | 
The Board Position after MCTS Agent Move is
| X | | O | |   | 
|   | | O | | X | 
|   | | X | |   | 
GAME is continuing

Starting Board Position is:
| X | | O | |   | 
|   | |   | | X | 
|   | | O | |   | 
The Board Position after MCTS Agent Move is
| X | | O | |   | 
|   | | X | | X | 
|   | | O | |   | 
GAME is continuing

Starting Board Position is:
| X | | X | | O | 
|   | |   | |   | 
| O | |   | |   | 
The Board Position after MCTS Agent Move is
| X | | X | | O | 
|   | | X | |   | 
| O | |   | |   | 
GAME is continuing

Starting Board Position is:
| X | | X | | O | 
|   | | O | |   | 
|   | |   | |   | 
The Board Position after MCTS Agent Move is
| X | | X | | O | 
|   | | O | |   | 
| X | |   | |   | 
GAME is continuing



**Opponent has made the First move and has occupied the centre square**

In [4]:
# Board Positions
# 0 - Empty
# 5 - X (Agent)
# 7 - O (Opponent)
print("Opponent has made the First move and has occupied the centre square")
# Opp - Center Square
game_boards = [[0,0,0,0,7,0,0,0,0],[0,0,0,5,7,7,0,0,0]]

# Testing when MCTS agent is one move away from win
for game_board in game_boards:
    base_node = tree_node(game_board,7)
    print("Starting Board Position is:")
    visualise_node_board(base_node)
    best_node = mcts(base_node,1000)
    print("The Board Position after MCTS Agent Move is")
    visualise_node_board(best_node)
    win = check_win(best_node)
    if win == 1:
        print("MCTS Agent Wins")
    elif win == 2:
        print("Opponent Wins")
    elif win == -2:
        print("The GAME is a TIE")
    else:
        print("GAME is continuing")
    print()

Opponent has made the First move and has occupied the centre square
Starting Board Position is:
|   | |   | |   | 
|   | | O | |   | 
|   | |   | |   | 
The Board Position after MCTS Agent Move is
| X | |   | |   | 
|   | | O | |   | 
|   | |   | |   | 
GAME is continuing

Starting Board Position is:
|   | |   | |   | 
| X | | O | | O | 
|   | |   | |   | 
The Board Position after MCTS Agent Move is
|   | |   | |   | 
| X | | O | | O | 
| X | |   | |   | 
GAME is continuing



# **MCTS vs Random Opponent**

In [5]:
# MCTS vs Random Opponent
# tie_factors = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
# for tie_factor in tie_factors:
wins = 0
tie = 0
loses = 0

# Playing 1000 Games MCTS vs Random Opponent
for i in range(1000):
    board = [0]*9
    board = random_opponent(board)
    base_node = tree_node(board,7)
    while(check_win(base_node) == -1):
        best_node = mcts(base_node,100)
        board = best_node.board
        board = random_opponent(board)
        base_node = tree_node(board,7)
    win = check_win(base_node)
    # Agent WIn
    if win == 1:
        wins += 1
    # Opp Win
    elif win == 0:
        loses += 1
    # Tie
    elif win == -2:
        tie += 1
print("MCTS vs Random Opponent")
print("Wins: ",wins)
print("Ties: ",tie)
print("Loses: ",loses)

MCTS vs Random Opponent
Wins:  764
Ties:  18
Loses:  218


# **MCTS vs Safe Opponent**

In [6]:
# MCTS vs SAFE Agent

# If Tie Factor = 0
# MCTS vs SAFE Opponent
# Total Games:  1000
# Wins:  203
# Ties:  193
# Loses:  604

# If Tie Factor = 1
# MCTS vs SAFE Opponent
# Total Games:  1000
# Wins:  146
# Ties:  705
# Loses:  149

# Function which gives a Safe Move as Output
def give_safe_move(board,i,j):
        out = -1
        if i == 0:
            if (j == 1 and board[2] == 0):
                out = 2
            elif(j == 2 and board[1] == 0):
                out = 1
            elif(j == 3 and board[6] == 0):
                out = 6
            elif(j == 4 and board[8] == 0):
                out = 8
            elif(j == 6 and board[3] == 0):
                out = 3
            elif(j == 8 and board[4] == 0):
                out = 4
        elif i == 1:
            if (j == 2 and board[0] == 0):
                out = 0
            elif(j == 4 and board[7] == 0):
                out = 7
            elif(j == 7 and board[4] == 0):
                out = 4
        elif i == 2:
            if (board[6] == 0 and j == 4):
                out = 6
            elif(board[8] == 0 and j == 5):
                out = 8
            elif(board[4] == 0 and j == 6):
                out = 4
            elif(board[5] == 0 and j == 8):
                out = 5
        elif i == 3:
            if (board[5] == 0 and j == 4):
                out = 5
            elif(board[4] == 0 and j == 5):
                out = 4
            elif(board[0] == 0 and j == 6):
                out = 0
        elif i == 4:
            if (board[3] == 0 and j == 5):
                out = 3
            elif(board[1] == 0 and j == 7):
                out = 1
            elif(board[0] == 0 and j == 8):
                out = 0
        elif i == 5:
            if (board[2] == 0 and j == 8):
                out = 2
        elif i == 6:
            if (board[8] == 0 and j == 7):
                out = 8
            elif(board[7] == 0 and j == 8):
                out = 7
        elif i == 7:
            if (board[6] == 0 and j == 8):
                out = 6
        return out

# Function which plays the Board with a Safe Move
def safe_opponent(board):
        l = []
        flag1 = 1
        flag2 = 1
        pos = -1
        for i in range(9):
            if board[i] == 7:
                l.append(i)
        if len(l) > 1:
            for a in range(len(l)):
                i = l[a]
                for b in range(a+1,len(l)):
                    # 12 13 14 15 17 19
                    # 23 25 28
                    # 357 369 375 396
                    # 456 465 471
                    # 564 582 591
                    # 693
                    # 789 798
                    # 897
                    j = l[b]
                    x = give_safe_move(board,i,j)
                    if x > -1:
                        pos = x
                        flag1 = 0
                        board[pos] = 7
                        return board
        if flag1:
            l = []
            for i in range(9):
                if board[i] == 5:
                    l.append(i)
            if len(l) > 1:
                for a in range(len(l)):
                    i = l[a]
                    for b in range(a+1,len(l)):
                        j = l[b]
                        x = give_safe_move(board,i,j)
                        if x > -1:
                            pos = x
                            flag2 = 0
                            board[pos] = 7
                            return board #"AGENT WIN"
        if flag2:
            l = []
            for i in range(9):
                if board[i] == 0:
                    l.append(i)
            pos = np.random.choice(l)
            board[pos] = 7
            return board #"CONTINUE"

wins = 0
tie = 0
loses = 0

# Playing 1000 Games MCTS vs Safe Agent
for i in range(1000):
    board = [0]*9
    board = safe_opponent(board)
    base_node = tree_node(board,7)
    while(check_win(base_node) == -1):
        best_node = mcts(base_node,100,tie_factor=0)
        board = best_node.board
        board = safe_opponent(board)
        base_node = tree_node(board,7)
    win = check_win(base_node)
    # Agent WIn
    if win == 1:
        wins += 1
    # Opp Win
    elif win == 0:
        loses += 1
    elif win == -2:
        tie += 1

print("MCTS vs SAFE Opponent")
print("Total Games: ",1000)
print("Wins: ",wins)
print("Ties: ",tie)
print("Loses: ",loses)

MCTS vs SAFE Opponent
Total Games:  1000
Wins:  226
Ties:  189
Loses:  585


# **MCTS vs MCTS**

# In the case of MCTS vs MCTS, the Agent who start first generally has more chance to win the game.

# They both try to win the Game and with High Tie Factor(=1), they end up as Ties,

In [7]:
# MCTS vs MCTS

wins = 0
tie = 0
loses = 0

# Function to Visualise a Board given a Board Pos
def visualize_board(board):
    for i in range(9):
        if board[i] == 0:
            print("|   |",end=" ")
        elif board[i] == 5:
            print("| X |",end=" ")
        else:
            print("| O |",end=" ")
        if i%3 == 2:
            print()

# Function to change Board Position from X to O and O to X
def change_board(board):
    new_board = board.copy()
    for i in range(len(new_board)):
        if new_board[i] == 5:
            new_board[i] = 7
        elif new_board[i] == 7:
            new_board[i] = 5
    return new_board

# Playing 1000 Games MCTS vs MCTS
for i in range(1000):
    board = [0]*9
    while(check_win_board(board) == -1):
        agent1_node = tree_node(board,7)
        agent1_move = mcts(agent1_node,200,tie_factor=1)
        board = agent1_move.board
        board = change_board(board)
        if check_win_board(board) != -1:
            break
        agent2_node = tree_node(board,7)
        agent2_move = mcts(agent2_node,200,tie_factor=1)
        board = agent2_move.board
        board = change_board(board)
    win = check_win_board(board)
    # Agent WIn
    if win == 1:
        wins += 1
    # Opp Win
    elif win == 0:
        loses += 1
    # TIE
    elif win == -2:
        tie += 1

print("MCTS vs MCTS")
print("Total Games: ",1000)
print("Wins: ",wins)
print("Ties: ",tie)
print("Loses: ",loses)

MCTS vs MCTS
Total Games:  1000
Wins:  0
Ties:  985
Loses:  15
